In [11]:
import pandas as pd 
import json
from openai import OpenAI

In [12]:
df = pd.read_pickle('personas_and_tweets.df.pkl')

## Add 2-shot to the persona

In [19]:
import random 

# Here is the updated dataframe with two randomly selected messages added to each user's persona,
# Function to update the persona with two random messages
def update_persona(group):
    messages = group['message'].tolist()
    if len(messages) >= 2:
        selected_messages = random.sample(messages, 2)
    else:
        selected_messages = messages
    examples_text = " Examples messages: '" + "'. '".join(selected_messages) + "'"
    group['personawithexamples'] = group['persona'].apply(lambda p: p + examples_text)
    return group

# Apply the function for each username
updated_df = df.groupby('username').apply(update_persona)


/var/folders/72/yhvs2cms3j9cxd1ln67f720h0000gn/T/ipykernel_29111/769202012.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  updated_df = df.groupby('username').apply(update_persona)


In [21]:
updated_df.to_pickle('personas_and_tweets.df.pkl')

## Generate messages using persona + 2-shot

In [ ]:
# Generate responses and store them in the column 'baseline'

openaiclient = OpenAI(api_key="xxx")
MODEL = 'gpt-4o'

def generate_tweet(persona,reply_to_message):

    prompt = f"Write a social media message in the style of the persona described below, in reply to the following message: '{reply_to_message}' \n\n Your persona: {persona}"
    
    messages = [{"role": "user", "content": f"{prompt}"}]

    print(messages)

    response = openaiclient.chat.completions.create(
        model=MODEL,
        temperature=0.1,
        messages=messages
    )

    result = ''.join(choice.message.content for choice in response.choices)
    return result 


df = pd.read_pickle('personas_and_tweets_baselinewithexamples.df.pkl')

# Ensure 'baseline' column exists
df['baseline'] = df.get('baseline', pd.NA)
i=0
# Loop over the dataframe
for index, row in df.iterrows():
    
    if row['training'] == 0 and pd.isna(row['baseline']):  # Skip already processed rows
        try:
            i+=1
            print(row['username'])            
            df.at[index, 'baseline'] = generate_tweet(row['persona'], row['reply_to'])
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue

        # Save progress every 10 rows
        if i % 10 == 0:
            df.to_pickle('personas_and_tweets_baselinewithexamples.df.pkl')
            print(f"Progress saved at row {index}")

            # print(len(df.loc[(df['training']==0)&(~df['baseline'].isna())])/len(df.loc[df['training']==0]))

    else:
        print('skip',end=", ")

# Final save after loop completion
df.to_pickle('personas_and_tweets_baselinewithexamples.df.pkl')
print("Processing complete and data saved.")

# generate_tweet(df.at[31000,'persona'],df.at[31000,'reply_to'])


## Generate messages using persona 

In [ ]:
# Generate responses and store them in the column 'baseline'

openaiclient = OpenAI(api_key="xxx")
MODEL = 'gpt-4o'

def generate_tweet(persona,reply_to_message):

    prompt = f"Write a social media message in the style of the persona described below, in reply to the following message: '{reply_to_message}' \n\n Your persona: {persona}"
    
    messages = [{"role": "user", "content": f"{prompt}"}]

    print(messages)

    response = openaiclient.chat.completions.create(
        model=MODEL,
        temperature=0.1,
        messages=messages
    )

    result = ''.join(choice.message.content for choice in response.choices)
    return result 


df = pd.read_pickle('personas_and_tweets.df.pkl')

# Ensure 'baseline' column exists
df['baselinewithexamples'] = df.get('baselinewithexamples', pd.NA)
i=0
# Loop over the dataframe
for index, row in df.iterrows():
    
    if row['training'] == 0 and pd.isna(row['baselinewithexamples']):  # Skip already processed rows
        try:
            i+=1
            print(row['username'])            
            df.at[index, 'baselinewithexamples'] = generate_tweet(row['personawithexamples'], row['reply_to'])
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue

        # Save progress every 10 rows
        if i % 10 == 0:
            df.to_pickle('personas_and_tweets.df.pkl')
            print(f"Progress saved at row {index}")

            # print(len(df.loc[(df['training']==0)&(~df['baseline'].isna())])/len(df.loc[df['training']==0]))

    else:
        print('skip',end=", ")

# Final save after loop completion
df.to_pickle('personas_and_tweets.df.pkl')
print("Processing complete and data saved.")

# generate_tweet(df.at[31000,'persona'],df.at[31000,'reply_to'])
